In [ ]:
%display latex

import numpy as np
import matplotlib.pyplot as plt

def electric_field(x, y, charges):
    """
    Docstring goes here
    """
    Ex, Ey = np.zeros_like(x), np.zeros_like(y)
    k = 8.99e9  # Coulomb's constant in N·m²/C²

    for (xq, yq, q) in charges:
        dx, dy = x - xq, y - yq
        r = np.linalg.norm([dx, dy], axis=0)
        r3 = r**3
        Ex += k * q * dx / r3
        Ey += k * q * dy / r3

    return Ex, Ey


def plot_field(charges, grid_size=20, normalize=True):
    x = np.linspace(-5, 5, grid_size)
    y = np.linspace(-5, 5, grid_size)
    X, Y = np.meshgrid(x, y)

    Ex, Ey = electric_field(X, Y, charges)
    magnitude = np.sqrt(Ex**2 + Ey**2)

    if normalize:
        Ex /= magnitude
        Ey /= magnitude

    plt.figure(figsize=(6, 6))
    plt.quiver(X, Y, Ex, Ey, magnitude, cmap='Blues', scale=20, norm='log')
    plt.colorbar(label='Field Strength')

    for (xq, yq, q) in charges:
        plt.scatter(xq, yq, c='red' if q > 0 else 'blue', s=100)

    plt.xlim(-5, 5)
    plt.ylim(-5, 5)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Electric Field')
    plt.show()

# Example charges: (x, y, charge)
charges = [(2, 2, 1e-9), (-2, -2, -1e-9)]
plot_field(charges)


In [ ]:
%display latex
from sage import all
import numpy as np

from functools import reduce

# Cartesian coordinates
x,y,z = var('x y z')

ε_0 = 8.8541878188e-12      # dielectric permittivity of vacuum
𝛋 = 1. / (4 * np.pi * ε_0)  # Coulomb's constant in N·m²/C²
total_charge = 1

chrgs = [
    (0,0,0),
    (1,0,0),
    (0,1,0),
    (1,1,0)
]

E = lambda _x, _y, _z, _dq, _chrgs: reduce(
    lambda a,b: simplify(a+b),
    [
        ((𝛋 * _dq) / (u.norm())^2) * u.normalized() if u.norm() > 0 else 0
        for u in [ 
            vector([_x,_y,_z]) - vector([dq_x, dq_y, dq_z]) # vector displacements
            for (dq_x, dq_y, dq_z) in _chrgs
        ]
    ]
)

p = plot_vector_field3d(
    ( 
       lambda x, y, z: E(_x=x, _y=y, _z=z, _dq=total_charge, _chrgs=chrgs)[0],
       lambda x, y, z: E(_x=x, _y=y, _z=z, _dq=total_charge, _chrgs=chrgs)[1],
       lambda x, y, z: E(_x=x, _y=y, _z=z, _dq=total_charge, _chrgs=chrgs)[2],
    ),
    (x,-2,2), (y,-2,2), (z,0,2), 
    center_arrows=True
)
show(p)


Graphics3d Object